# Retreival System

> Aim: create a database containing previous puzzles (with there solutions) and create a retreival system that given a new puzzle finds similar puzzles and returns the (best) solutions for the similar puzzles.

In [3]:
import os
import sys

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.getcwd(), '../src/')
print(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

from models.gemini_model import GeminiLanguageModel

/home/twanh/workspace/thesis/thesis-advent-of-agents/experiments/../src/


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
# Create a model that can be used
model = GeminiLanguageModel(
    api_key=os.getenv("GEMINI_API_KEY"),
    model_name='gemini-2.0-flash'
)

In [5]:
# Load all puzzles
# puzzles are stored in a directory structure [year]/[day].txt
PUZZLES_PATH = '/home/twanh/workspace/thesis/puzzles/auto'


In [24]:
# Setup the postgres database
from pgvector.psycopg import register_vector
import psycopg

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

conn.execute('CREATE EXTENSION IF NOT EXISTS vector')

register_vector(conn)

conn.execute('''
-- Puzzles table
CREATE TABLE puzzles (
  id SERIAL PRIMARY KEY,
  year INT NOT NULL,
  day INT NOT NULL,
  full_description TEXT,
  problem_statement TEXT,
  keywords TEXT,
  underlying_concepts TEXT,
  embedding VECTOR(1536),  -- OpenAI embedding dimension (small model)
  UNIQUE(year, day)  -- Ensure year/day combinations are unique
);

-- Create an index on the vector column for fast nearest-neighbor search
CREATE INDEX idx_embedding ON puzzles
  USING ivfflat (embedding vector_l2_ops) WITH (lists = 100);

-- Solutions table with a foreign key to puzzles
CREATE TABLE solutions (
  id SERIAL PRIMARY KEY,
  puzzle_id INT NOT NULL,
  code TEXT NOT NULL,
  author VARCHAR(255),
  source VARCHAR(255),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (puzzle_id) REFERENCES puzzles(id) ON DELETE CASCADE
);

-- Index on puzzle_id for faster joins
CREATE INDEX idx_solutions_puzzle_id ON solutions(puzzle_id);
''')
conn.close()

DuplicateTable: relation "puzzles" already exists

In [7]:
# Get all the paths for the puzzles
def get_puzzle_paths(puzzles_path):
    puzzle_paths = []
    for year in os.listdir(puzzles_path):
        year_path = os.path.join(puzzles_path, year)
        if os.path.isdir(year_path):
            for day in os.listdir(year_path):
                day_path = os.path.join(year_path, day)
                if os.path.isfile(day_path):
                    if day_path.endswith('.txt'):
                        puzzle_paths.append(day_path)
    return puzzle_paths
puzzle_paths = get_puzzle_paths(PUZZLES_PATH)
print(puzzle_paths)
print(f"Found {len(puzzle_paths)} puzzle files.")

['/home/twanh/workspace/thesis/puzzles/auto/2015/day_2_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_7_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_3_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_11_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_19_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_15_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_14_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_9_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_5_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_16_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_20_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_8_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_1_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_23_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_6_

In [29]:
from dataclasses import dataclass

@dataclass
class PuzzleData:
    full_description: str
    year: int
    day: int
    problem_statement: str
    keywords: list[str]
    underlying_concepts: list[str]
    
    def __getitem__(self, item):
        return getattr(self, item)


In [18]:
import re
from pprint import pformat
from agents.pre_processing_agent import PreProcessingAgent
from core.state import MainState
from utils.util_types import Puzzle

def extract_year_day(puzzle_path: str) -> tuple[int,int]:


    pattern = r".*/(\d{4})/day_(\d+)_part_1\.txt"
    match = re.match(pattern, puzzle_path)
    if match:
        year, day = match.groups()
        return (year, day)
        

    print(f"Error: could not extract year,day from: {puzzle_path}")
    return 0, 0


def pre_process_puzzle(puzzle_path: str) -> PuzzleData:

    with open(puzzle_path, 'r') as puzzle_file:
        puzzle_desc = puzzle_file.read()

    year, day = extract_year_day(puzzle_path)

    puzzle = Puzzle(
        description=puzzle_desc,
        day = day,
        year=year,
        solution=None
    )

    state = MainState(puzzle=puzzle)

    pre_agent = PreProcessingAgent(agent_name='pre-processing', model=model)

    updated_state = pre_agent.process(state)

    pformat(updated_state)

    return PuzzleData(
        full_description=updated_state.puzzle.description,
        year = updated_state.puzzle.year,
        day = updated_state.puzzle.day,
        problem_statement=updated_state.problem_statement,
        underlying_concepts=updated_state.underlying_concepts,
        keywords=updated_state.keywords,
    )


pre_processed_puzzles = [pre_process_puzzle(puzzle_path) for puzzle_path in puzzle_paths[:5]]

2025-04-15 10:15:14.105 | DEBUG    | agents.pre_processing_agent:process:21 - Preprocessing agent prompt: You are a pre-processing agent. Your task is to process the following Advent of Code puzzle description so that the output can be stored in a RAG (Retrieval-Augmented Generation) database and later used by a planning agent. Follow these steps precisely:

------------------------------------------------------------
Step 1: Detailed Extraction
------------------------------------------------------------
Extract every technical detail from the puzzle and separate it from the narrative. Specifically, please extract:
  - Core Problem Statement:
      - A concise description of the computational task (exclude story or decorative language).
  - Input Specifications:
     - Detailed descriptions of the expected input format, data types, ranges, and constraints.
  - Output Specifications:
      - Clear requirements on what the solution must produce (format, limits, etc.).
  - Test cases inp

In [21]:
import pprint
pprint.pprint(pre_processed_puzzles)


[PuzzleData(full_description='## \\--- Day 2: I Was Told There Would Be No '
                             'Math ---\n'
                             '\n'
                             'The elves are running low on wrapping paper, and '
                             'so they need to submit an order for more. They '
                             'have a list of the dimensions (length `l`, width '
                             '`w`, and height `h`) of each present, and only '
                             'want to order exactly as much as they need.\n'
                             '\n'
                             'Fortunately, every present is a box (a perfect '
                             '[right rectangular '
                             'prism](https://en.wikipedia.org/wiki/Cuboid#Rectangular_cuboid)), '
                             'which makes calculating the required wrapping '
                             'paper for each gift a little easier: find the '
                             'su

In [50]:
import numpy as np
import dataclasses
from openai import OpenAI

WEIGHTS = {
    'full_description': 0.1,
    'problem_statement': 0.3,
    'underlying_concepts': 0.4,
    'keywords': 0.2,
}


embedding_model = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def create_embedding(text: str):

    response = embedding_model.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )

    return response.data[0].embedding


def compute_weighted_embedding(puzzle_data: PuzzleData, weights: dict[str, float]) -> list[float]:

    puzzle_data = dataclasses.asdict(puzzle_data)
    # THis assumes that all the fields have a weight if they need to be included in the compsite embedding
    embeddings = {}
    for field in weights:
        embeddings[field] = np.array(create_embedding(puzzle_data[field]))


    composite_embedding = np.zeros_like(list(embeddings.values())[0])
    for field, weight in weights.items():
        weighted_embedding = weight * embeddings[field]
        composite_embedding += weighted_embedding

    # Normalize the embedding
    norm = np.linalg.norm(composite_embedding)
    if norm > 0:
        composite_embedding  = composite_embedding / norm

    return composite_embedding.tolist()




In [ ]:
def add_puzzle_to_db(puzzle_data: PuzzleData, conn) -> int:

    embedding = compute_weighted_embedding(puzzle_data, WEIGHTS)

    with conn.cursor() as cur:

        query = """
        INSERT INTO puzzles (
            year, day, full_description, problem_statement, 
            keywords, underlying_concepts, embedding
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        RETURNING id;
        """
        
        cur.execute(query, (
            puzzle_data.year,
            puzzle_data.day,
            puzzle_data.full_description,
            puzzle_data.problem_statement,
            puzzle_data.keywords,
            puzzle_data.underlying_concepts,
            embedding
        ))
        
        puzzle_id = cur.fetchone()[0]
        conn.commit()
    
        return puzzle_id

In [51]:

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

for puzzle in pre_processed_puzzles:

    pid = add_puzzle_to_db(puzzle, conn)
    print(f"Added puzzle to db with puzzle_id: {pid}")

AttributeError: 'PuzzleData' object has no attribute 'concepts'